In [24]:
print 1

1


In [124]:
import sys
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/semi_supervised/')
import coreg
reload(coreg)
import rasco
reload(rasco)
import trireg
reload(trireg)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ordinal/')
import simple
reload(simple)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ensemble/')
import stacking
reload(stacking)

sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/neighbors/')
import rpfnn
reload(rpfnn)
import ann
reload(ann)

import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

import transformers as tforms
reload(tforms)
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union

import metrics
reload(metrics)
from sklearn.cross_validation import StratifiedKFold, train_test_split

from sklearn.base import clone
from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet, Ridge, Lasso, SGDRegressor, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier, DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.lda import LDA
from sklearn.qda import QDA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.isotonic import IsotonicRegression
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, MaxoutDense, Reshape
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.optimizers import Adadelta, Adagrad, Adam, RMSprop
from keras.layers.advanced_activations import ParametricSoftplus, PReLU

from svmlight_loader import dump_svmlight_file, load_svmlight_file

from collections import Counter
import minirank as mr

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from time import time

import cPickle as pickle

def wgmean(x, w):
    return np.exp(np.sum(w*np.log(x), axis=1) / np.sum(w, axis=1))

import logging 



In [125]:
# Load Data custom
train_pd  = pd.read_pickle('saved/train_pd_custom.p')
test_pd  = pd.read_pickle('saved/test_pd_custom.p')

labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')

In [5]:
# mapped_train, _y = load_svmlight_file('saved/mapped2000_train.libsvm')
# mapped_test, _y = load_svmlight_file('saved/mapped2000_test.libsvm')

mapped_train, _y = load_svmlight_file('saved/mapped1000_train_79.libsvm')
mapped_test, _y = load_svmlight_file('saved/mapped1000_test)79.libsvm')

X_2000mean_train = mapped_train.todense()
X_2000mean_test = mapped_test.todense()
print X_2000mean_train.shape
print X_2000mean_test.shape

(50999, 2000)
(51000, 2000)


In [126]:
train = np.array(train_pd)
test = np.array(test_pd)

X_train = train.astype(float)
X_test = test.astype(float)
y_train = np.array(labels)
y_train_orig = np.array(labels)

try:
    X_train = np.c_[X_train, X_tsne2_26_train]
    X_test = np.c_[X_test, X_tsne2_26_test]
except:
    pass

try:
    X_train = np.c_[X_train, X_2000mean]
except:
    pass

holdout = False
if holdout:
    X_train, X_hold, \
    y_train, y_hold, \
    y_binned, y_binned_hold \
    = train_test_split(
        X_train, y_train, y_binned, 
        test_size=0.2, random_state=0)

# """
pipe_x = make_pipeline(
    make_union(
        tforms.IdentityTformer(),
    ),
    StandardScaler(),
)
pipe_y = make_pipeline(
#     tforms.IdentityTformer(),
    tforms.GeneralTformer(lambda x: np.log(x)),
)
pipe_x.fit(np.r_[X_train, X_test])
pipe_y.fit(y_train)

X_train = pipe_x.transform(X_train)
X_test = pipe_x.transform(X_test)
try:
    X_hold = pipe_x.transform(X_hold)
except:
    pass

y_train = pipe_y.fit_transform(y_train)

# """

print 'y_train', y_train.shape
print 'X_train', X_train.shape
print 'X_test', X_test.shape
try:
    print 'X_hold', X_hold.shape
except:
    pass
print len(np.unique(y_train))
print len(np.unique(y_binned))
print y_binned.shape
print type(X_train[0][0])

y_train (50999,)
X_train (50999, 79)
X_test (51000, 79)
X_hold 50
5
(50999,)
<type 'numpy.float64'>


In [7]:
# X_nn_train = np.c_[X_2000mean_train, X_train]
# X_nn_test = np.c_[X_2000mean_test, X_test]

X_nn_train = np.c_[X_2000mean_train, X_train]
X_nn_test = np.c_[X_2000mean_test, X_test]

print X_nn_train.shape
print X_nn_test.shape

(50999, 2129)
(51000, 2129)


In [127]:
X_all = np.r_[X_train, X_test]
y_binned_all = np.r_[y_binned-1, np.nan*np.ones(len(X_test))]
print X_all.shape
print y_binned_all.shape

(101999, 79)
(101999,)


# DEFINE THE MODELS
## BASE MODELS

### NN (using sofia kmeans cluster distance maps)

In [128]:
n_feats = X_nn_train.shape[1]
drop_prob = 0.6

model = Sequential()

hidden_size = 1024
model.add(Dense(n_feats, hidden_size))
model.add(Dropout(0.2))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, 1))
model.add(Activation('linear'))

loss_type = 'msle'

opt = RMSprop(lr=0.0005, rho=0.75, epsilon=1e-6)

nn_reg = KerasRegressor(model, optimizer=opt, loss=loss_type,
                        train_batch_size=1024*4, test_batch_size=1024*16,
                        nb_epoch=500)

NameError: name 'X_nn_train' is not defined

### NN for 79 feats as normal

In [129]:
n_feats = X_train.shape[1]
drop_prob = 0.5

model = Sequential()

# # for 79 feats
hidden_size = 256
model.add(Dense(n_feats, hidden_size))
model.add(Dropout(0.2))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, 1))
model.add(Activation('linear'))

loss_type = 'mse'

opt = RMSprop(lr=0.001, rho=0.75, epsilon=1e-6)

nn_reg = KerasRegressor(model, optimizer=opt, loss=loss_type,
                        train_batch_size=1024*4, test_batch_size=1024*16,
                        nb_epoch=500)

### GBM XGBOOST

In [130]:
gbm_reg = xgb.XGBRegressor(
    objective="reg:linear",
    n_estimators=2800,
    learning_rate=0.005,
#     gamma=0.0,
    max_depth=5,
    min_child_weight=6,
#     max_delta_step=10,
    subsample=0.7,
    colsample_bytree=0.7,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=7,
    seed=322,
    silent=True,
)

gbm_bin = xgb.XGBClassifier(
    objective="binary:logistic",
    n_estimators=150,
    learning_rate=0.05,
#     gamma=0.0,
    max_depth=5,
    min_child_weight=6,
    subsample=0.7,
    colsample_bytree=0.7,
    base_score=0.5,
    nthread=7,
    seed=322,
    silent=True,
)

gbm_cat = xgb.XGBClassifier(
    objective="multi:softmax",
    n_estimators=150,
    learning_rate=0.05,
#     gamma=0.0,
    max_depth=5,
    min_child_weight=6,
    subsample=0.7,
    colsample_bytree=0.7,
    base_score=0.5,
    nthread=7,
    seed=322,
    silent=True,
)



### RF

In [131]:
rf_reg = RandomForestRegressor(n_jobs=-1, **{'max_leaf_nodes': None, 
                                         'bootstrap': False, 
                                         'min_samples_leaf': 8, 
                                         'n_estimators': 150, 
                                         'min_samples_split': 8, 
                                         'min_weight_fraction_leaf': 0.25, 
                                         'max_features': 'sqrt', 
                                         'max_depth': 12} )

### Other crappy regressors

In [132]:
lin_reg = LinearRegression()
ridge_reg = Ridge()

### Custom weird stuff

In [133]:
# h = [GaussianNB() for _ in range(16)] + \
#     [DecisionTreeClassifier(max_depth=6, min_samples_split=5, class_weight='auto') for _ in range(4)] + \
#     [SGDClassifier(n_iter=20, loss='modified_huber', class_weight='auto', n_jobs=1) 
#      for _ in range(2)] + \
#     [ExtraTreeClassifier(max_depth=6, min_samples_split=6, class_weight='auto')
#      for _ in range(4)]
    
h = [GaussianNB() for _ in range(16)] + \
    [DecisionTreeClassifier(max_depth=6, min_samples_split=5, class_weight='auto') for _ in range(4)]

rasco_mix = rasco.Rasco(h,
                        feat_ratio=0.6,
                        n_estimators=20, max_iters=20,
                        n_xfer=50,
                        verbose=True,
                        n_jobs=-1)

rasco_gbm = rasco.Rasco([clone(gbm_bin) for _ in range(16)],
                       feat_ratio=0.6,
                       n_estimators=16, max_iters=10,
                       n_xfer=200,
                       verbose=True,
                       n_jobs=1)

rasco_gbm_cat = rasco.Rasco([clone(gbm_cat) for _ in range(8)],
                       feat_ratio=0.6,
                       n_estimators=8, max_iters=20,
                       n_xfer=100,
                       verbose=True,
                       n_jobs=1)

# ord_rasco = simple.SimpleOrdinalClassifier(rasco_mix, n_jobs=1)
ord_rasco = simple.SimpleOrdinalClassifier(rasco_gbm, n_jobs=1)

ord_gaussnb = simple.SimpleOrdinalClassifier(GaussianNB(), n_jobs=-1)


INFO:rasco:RASCO Init
INFO:rasco:RASCO Init
INFO:rasco:RASCO Init


## META MODELS

### NN META

In [135]:
n_feats = 5
drop_prob = 0.5

model_meta = Sequential()
hidden_size = 128
model_meta.add(Dense(n_feats, hidden_size))
model_meta.add(Dropout(0.2))
model_meta.add(ParametricSoftplus(hidden_size))
model_meta.add(BatchNormalization((hidden_size,)))
model_meta.add(Dropout(drop_prob))
model_meta.add(Dense(hidden_size, hidden_size))
model_meta.add(ParametricSoftplus(hidden_size))
model_meta.add(BatchNormalization((hidden_size,)))
model_meta.add(Dropout(drop_prob))
model_meta.add(Dense(hidden_size, hidden_size))
model_meta.add(ParametricSoftplus(hidden_size))
model_meta.add(BatchNormalization((hidden_size,)))
model_meta.add(Dropout(drop_prob))
model_meta.add(Dense(hidden_size, 1))
model_meta.add(Activation('linear'))

loss_type_meta = 'msle'

opt_meta = RMSprop(lr=0.0005, rho=0.75, epsilon=1e-6)

nn_meta = KerasRegressor(model=model_meta, optimizer=opt_meta, loss=loss_type_meta,
                        train_batch_size=1024*4, test_batch_size=1024*16,
                        nb_epoch=210)

# THE STACK

In [138]:
fh = logging.FileHandler('/tmp/lmg_%d.log' % int(time()))

stack = stacking.Stacking([
        #            Ind   Output Dim
        gbm_reg,     # 0     1
        rf_reg,      # 1     1
        lin_reg,     # 2     1
        ord_rasco,   # 3     1/5
        nn_reg,      # 4     1
    ],
#                           nn_meta,
#                           meta_gbm,
                          LinearRegression(),
                          fit_params={
        'base3_X': 'X_all',
        'base3_y': 'y_binned_all',
    },
                          pred_params={
        'base3_weighted': True,
    },
                          extra_data={
        'y_binned': y_binned-1,
        'X_all': X_all,
        'y_binned_all': y_binned_all,
    },
                          include_orig_feats=False,
                          use_probs=False,
                          cv=5,
                          verbose=1,
                          save_level0_out=True,
                          log_handler=fh,
                         )

INFO:stacker:Stacker Init
DEBUG:stacker:Extra Data: ['y_binned', 'X_all', 'y_binned_all']


# TRAINING

In [ ]:
%pdb off
stack.fit(X_train, y_train)

DEBUG:stacker:===[Fit & predict fold 0 for base models]===
DEBUG:stacker:[XGBRegressor]
DEBUG:stacker:	Time fit:		78.8271 s
DEBUG:stacker:	Time predict:		0.459135 s
DEBUG:stacker:[RandomForestRegressor]
DEBUG:stacker:	Time fit:		4.7068 s
DEBUG:stacker:	Time predict:		0.210986 s
DEBUG:stacker:[LinearRegression]
DEBUG:stacker:	Time fit:		0.472468 s
DEBUG:stacker:	Time predict:		0.0015831 s
DEBUG:stacker:[SimpleOrdinalClassifier]
DEBUG:stacker:	X data: X_all
DEBUG:stacker:	y data: y_binned_all
DEBUG:rasco:Best candidate: pred_class=[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] | Prob: (200, (0.82294124, 0.88901913), 0.83759534, 0.0001

# PREDICTION

In [ ]:
y_pred_test = stack.predict(X_test)

In [ ]:
plt.hist(y_pred_test, alpha=0.8)
plt.hist(y_train, alpha=0.3)

# SUBMISSION

In [ ]:
preds_pd = pd.DataFrame({"Id": test_ind, "Hazard": y_pred_test})
preds_pd = preds_pd.set_index('Id')
preds_pd.to_csv('submissions/give_up.csv')

# MOAR SANDBOXING

In [52]:
params = {}
params["objective"] = "binary:logistic"
params["eta"] = 0.05
params["min_child_weight"] = 6
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
# params["max_delta_step"] = 1
params["silent"] = 1
params["max_depth"] = 5
# params["scale_pos_weight"] = 1.0
# params["gamma"] = 0

plst = list(params.items())

max_rounds = 5000

skf = StratifiedKFold(y_binned, n_folds=8,
                      shuffle=True,
                      random_state=np.random.randint(0,100))
qq_scores = []
for train, valid in skf:
    X_train_k = X_train[train]
    X_valid_k = X_train[valid]    
    y_train_k = y_binned[train] > 3
    y_valid_k = y_binned[valid] > 3
    
    y_valid_k_orig = y_train_orig[valid]


    #create a train and validation dmatrices 
    xgtrain = xgb.DMatrix(X_train_k, label=y_train_k)
    xgval = xgb.DMatrix(X_valid_k, label=y_valid_k)

    #train using early stopping and predict
    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model = xgb.train(plst, xgtrain, max_rounds, watchlist, 
                      early_stopping_rounds=100,
                      verbose_eval=False)

    preds_val = model.predict(xgval)

    s = metrics.normalized_gini(y_valid_k_orig, preds_val)
    compare = metrics.normalized_gini(y_valid_k_orig, y_valid_k)
    print 'xgb:', s
    print 'compare:', compare
    qq_scores.append(s)
    
#     break

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[217]	train-error:0.194523	val-error:0.201035

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[68]	train-error:0.200506	val-error:0.203387



xgb: 0.383455332761
compare: 0.66955245128
xgb:

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[173]	train-error:0.195729	val-error:0.202321



 0.351025279465
compare: 0.665826502033
xgb:

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[255]	train-error:0.194339	val-error:0.197930



...

 0.363570025434
compare: 0.674246917105
xgb:

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[2]	train-error:0.201358	val-error:0.205744



 0.365592359058
compare: 0.673994123422
xgb:

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[339]	train-error:0.191902	val-error:0.202291



 0.378550003724
compare: 0.67131354438
xgb: 0.394293036864
compare: 0.677817610001


In [32]:
150 iter

xgb: 0.349063528672

    xgb: 0.348477999514
d3
        xgb: 0.359185167985

array([1, 2, 3, 4, 5])

# More CV Sandbox

In [121]:
scores = []
scores_base = []
lols = []
n_reps = 1
k = 3

# clfs = [rasco_gbm_cat]
# clfs = [simple.SimpleOrdinalClassifier(gbm_bin, n_jobs=1)]

gg = rasco.Rasco([clone(gbm_bin) for _ in range(32)],
                       feat_ratio=0.6,
                       n_estimators=32, max_iters=10,
                       n_xfer=100,
                       verbose=True,
                       n_jobs=1)
ord_gg = simple.SimpleOrdinalClassifier(gg, n_jobs=1)
clfs = [ord_gg]


for reps in range(n_reps):
    skf = StratifiedKFold(y_binned, n_folds=k,
                          shuffle=True,
                          random_state=np.random.randint(0,100))
    for ii, (train, valid) in enumerate(skf):
        print 'Fold %d' % ii,
        X_train_k, X_valid_k = X_train[train], X_train[valid]
        y_train_k, y_valid_k = y_train[train], y_train[valid]
        y_train_binned_k, y_valid_binned_k = y_binned[train], y_binned[valid]
        y_valid_k_orig = y_train_orig[valid]
        
        X_all = np.r_[X_train_k, X_test]
        y_all = np.r_[y_train_k, np.nan*np.ones(len(X_test))]
        y_binned_all = np.r_[y_train_binned_k, np.nan*np.ones(len(X_test))]
        
        tic = time()

        for clf in clfs:
            clf.fit(X_all, y_binned_all-1)
#             clf.fit(X_train_k, y_train_binned_k-1)

        
        toc = time() - tic
        print 'Train time: %2.3f s\t' % toc,
        tic = time()

        
        valid_preds = [clf.predict(X_valid_k, weighted=True) for clf in clfs]
        
        
        score = [metrics.normalized_gini(y_valid_k_orig, v) for v in valid_preds]
        
        toc_pred = time() - tic
        print 'Pred time: %2.3f s\t' % toc_pred, 
        
        print 'Score', score
        scores.append(score)
        break
        
        
print "done"
print 'Score:', np.array(scores).mean(axis=0)

INFO:rasco:RASCO Init
DEBUG:rasco:Best candidate: pred_class=[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] | Prob: (100, (0.82662004, 0.87535834), 0.83903646, 0.00011442679, 1.1383524693870781, 0.9051935558129394)
DEBUG:rasco:L-shape: (34098, 79) 	 U-shape: (50900, 79)
DEBUG:rasco:Iter #0 time: 81.6941
DEBUG:rasco:Best candidate: pred_class=[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] | Prob: (100, (0.82104379, 0.85152328), 0.82902962, 3.6760011e-05, 0.9190436389757458, 0.5595329089297025)
DEBUG:rasco:L-shape: (34198, 79) 	 U-shape: (50800, 79)
DEBUG:rasco:Iter #1 time: 82.0159
DEBUG:rasco:Best candidate: pred_class=[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

Fold 0

KeyboardInterrupt: 

In [91]:
valid_preds = [clf.predict(X_valid_k) for clf in clfs]

score = [metrics.normalized_gini(y_valid_k_orig, v) for v in valid_preds]

toc_pred = time() - tic
print 'Pred time: %2.3f s\t' % toc_pred, 

print 'Score', score
scores.append(score)

Pred time: 1968.875 s	Score [-0.010252387650191009]


In [96]:
from collections import Counter
Counter(valid_preds[0])


Counter({0: 16995, 2: 6})

In [98]:
probs = clf.predict_proba(X_valid_k)

In [114]:
pd.DataFrame(np.c_[probs[:10,:], y_valid_binned_k[:10]])


,0,1,2,3,4,5
0,0.373863,0.172267,0.247278,0.157619,0.048973,1
1,0.355497,0.164078,0.273147,0.156618,0.050660,3
2,0.359538,0.175760,0.256977,0.151940,0.055785,1
3,0.365223,0.154651,0.254370,0.170097,0.055659,1
4,0.387085,0.152515,0.246580,0.157921,0.055898,1
5,0.385071,0.168589,0.264334,0.135865,0.046141,1
6,0.372401,0.141727,0.275748,0.153044,0.057080,5
7,0.387455,0.144027,0.268762,0.150189,0.049567,5
8,0.415649,0.147575,0.250130,0.137614,0.049032,3
9,0.368009,0.159706,0.256766,0.167969,0.047550,1
